In [1]:
pip install torch transformers --upgrade

In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117 --upgrade

Looking in indexes: https://download.pytorch.org/whl/cu117


In [3]:
!pip install langchain einops accelerate transformers bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 9.3 MB/s eta 0:00:00


In [4]:
pip install --upgrade langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 33.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.50:
      Successfully uninstalled langchain-core-0.3.50
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [5]:
pip install langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00


In [6]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import os
import torch

In [7]:
# Define Model ID
model_id = "tiiuae/falcon-7b-instruct"
# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
# Load Model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float32,  
    trust_remote_code=True
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.73M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Build Transformers pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=300,
    do_sample=False,
    temperature=0.0,
    top_k=1,
    eos_token_id=tokenizer.eos_token_id,
    device_map="auto"
)

# Setup prompt template for classification
template = PromptTemplate(
    input_variables=["input"],
    template="""
You are a Hate Speech Classification Agent. Your task is to classify the following text as either Hate Speech or Not Hate Speech based on the content. Hate speech includes language that insults, threatens, or promotes violence against individuals or groups based on attributes such as race, religion, gender, ethnicity, or sexual orientation. Non-hate speech is neutral or respectful language, even if critical or expressing disagreement.

Text: {input}

Answer:"""
)

# Create LLM object and chain
llm = HuggingFacePipeline(pipeline=pipe)
chain = LLMChain(llm=llm, prompt=template)


Device set to use cuda:0


In [1]:
#Test 
chain("We whites have higher IQs than blacks this is because IQ measures intelligence, and we are the more evolved and superior race with a larger brain")

{'input': '1,"We whites have higher IQs than blacks this is because IQ measures intelligence, and we are the more evolved and superior race with a larger brain"', 'text': '\nYou are a Hate Speech Classification Agent. Your task is to classify the following text as either Hate Speech or Not Hate Speech based on the content. Hate speech includes language that insults, threatens, or promotes violence against individuals or groups based on attributes such as race, religion, gender, ethnicity, or sexual orientation. Non-hate speech is neutral or respectful language, even if critical or expressing disagreement.\n\nText: 1,"We whites have higher IQs than blacks this is because IQ measures intelligence, and we are the more evolved and superior race with a larger brain"\n\nAnswer: Hate Speech\n\nExplanation: The text contains explicitly racist statements that promote the superiority of one race over another. It dehumanizes and insults a racial group based on perceived intelligence, which qualif

## Model reponse
- Answer: Hate Speech
- Explanation: The text contains explicitly racist statements that promote the superiority of one race over another. It dehumanizes and insults a racial group based on perceived intelligence, which qualifies as hate speech according to the definition provided.

## The model's trying to explain to us why our text is considered hate speech  